In [64]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pyproj
import geopandas as gpd
from shapely.geometry import Point
from sklearn.linear_model import Lasso,LassoCV,Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [74]:
for_temple = pd.read_csv("public_quantity_distance.csv")
for_temple['最近金融機構距離']
for_temple['最近廟宇距離']

# concat_data = pd.read_csv("new_concat_data.csv").drop(['最近金融機構距離','最近廟宇距離'], axis=1).reset_index(drop=True)
concat_data = pd.read_csv("new_concat_data.csv").reset_index(drop=True)
origin_concat_data = pd.read_csv("concat_data.csv")
origin_concat_data = origin_concat_data[11751:].drop(['geometry','縱坐標','橫坐標'], axis=1).reset_index(drop=True)
origin_concat_data['最近金融機構距離'] = for_temple['最近金融機構距離']
origin_concat_data['最近廟宇距離'] = for_temple['最近廟宇距離']

new_concat_data = pd.concat([concat_data, origin_concat_data], ignore_index=True)

In [75]:
# concat_data = new_concat_data[[
#     '縣市', '鄉鎮市區', '路名', '主要用途', '主要建材', '建物型態', '使用分區', '備註', '單價',
#     'lng', '最近航空站', '最近幼稚園、托兒所',
#     '屋齡', '最近一般公園', '最近碼頭', '最近大型購物中心', 
#     '最近超級市場', '最近大學及研究所', '總樓層數',
#     '最近國中', '最近生活百貨量販',
#     '最近捷運站', '最近電力資源', '最近高鐵站',
#     '最近產業用水及設施', '最近飯店', '最近觀光遊憩設施', 
#     '最近主要道路建設', 'lat', '最近里鄰公園', '最近娛樂設施', 
#     '國中(高)', '國中(中)', '最近火車站', '車位面積', 
#     '最近百貨公司', '陽台面積', '移轉層次', '最近停車場地', 
#     '最近醫療保健', '最近加油站', 
#     '土地面積', '主建物面積', '建物面積', '最近工廠', 
#     '最近傳統市場', '附屬建物面積', '國中(下)', 
#     '最近高中', '最近商店', '最近專科', '最近國小', 
#     '最近廟宇距離', '最近金融機構距離'
# ]]
concat_data = new_concat_data[[
    '縣市', '鄉鎮市區', '路名', '主要用途', '主要建材', '建物型態', '使用分區', '備註', '單價', 
    '車位面積', '陽台面積', '土地面積', '主建物面積', '建物面積', '附屬建物面積',
    'lng', '最近航空站',
    '屋齡', '最近一般公園', '最近碼頭', '最近大型購物中心', 
    '最近超級市場', '總樓層數',
    '最近國中', '最近生活百貨量販',
    '最近捷運站', '最近電力資源', '最近高鐵站',
    '最近產業用水及設施', '最近飯店', '最近觀光遊憩設施', 
    '最近主要道路建設', 'lat', '最近里鄰公園',
    '國中(高)', '國中(中)', '最近火車站', 
    '最近百貨公司', '移轉層次', '最近停車場地', 
    '最近醫療保健', '最近加油站',
    '最近工廠', '最近傳統市場',
    '最近商店', '最近專科',
    '最近廟宇距離', '最近金融機構距離'
    ]]

In [76]:
z_scaler = preprocessing.StandardScaler()
concat_data.iloc[:, 9:] = pd.DataFrame(z_scaler.fit_transform(concat_data.iloc[:, 9:]))
concat_data = pd.get_dummies(concat_data, columns=['縣市','鄉鎮市區','路名','主要用途','主要建材','建物型態','使用分區','備註'])

/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_24326/1780255234.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concat_data.iloc[:, 9:] = pd.DataFrame(z_scaler.fit_transform(concat_data.iloc[:, 9:]))


In [77]:
Y_train = concat_data[['單價']].dropna(subset=['單價'])
X_train = concat_data.dropna(subset=['單價']).drop(['單價'], axis=1)
X_test = concat_data[concat_data['單價'].isna()].drop(['單價'], axis=1)

In [78]:
rf_regressor = RandomForestRegressor()
result = rf_regressor.fit(X_train.values, Y_train.values)

/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_24326/3776739804.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  result = rf_regressor.fit(X_train.values, Y_train.values)


In [79]:
predictions = result.predict(X_test.values)

In [81]:
predictions

array([1.89245799, 1.63874326, 2.71197155, ..., 1.94801664, 2.94619859,
       2.07520192])

In [82]:
result = pd.read_csv("public_submission_template.csv")
result['predicted_price'] = predictions
result.to_csv("./result.csv", index=False)